# Coloring

## Prerequisites
Import the required modules and run a few models with a small orbit library.

In [ ]:
import numpy as np
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
import cmasher

from vorbin.voronoi_2d_binning import voronoi_2d_binning

import dynamite as dyn

In [ ]:
print('DYNAMITE version', dyn.__version__)
print('    installed at ', dyn.__path__)  # Uncomment to print the complete DYNAMITE installation path

fname = 'FCC167/FCC167.yaml'
c = dyn.config_reader.Configuration(fname, reset_logging=True)
# c = dyn.config_reader.Configuration(fname, reset_logging=True, reset_existing_output=True)
# _ = dyn.model_iterator.ModelIterator(c)

## Voronoi orbit binning
Before binning the orbits, let's have a look at the $r$ - $\lambda_z$ phase space.

In [ ]:
# Number of desired r and lambda_z bins
nr = 6
nl = 7

In [ ]:
plotter = dyn.plotter.Plotter(c)

In [ ]:
# NOTE: when using force_lambda_z=True, then the titles of the orbit-distribution plots are incorrect: all orbits are shown in this distribution - not only short-axis tubes!
fig1 = plotter.orbit_distribution(model=None, minr=None, maxr=None, r_scale='log', nr=nr, nl=nl,
                                  orientation='vertical', subset='short', force_lambda_z=True)
fig2 = plotter.orbit_distribution(model=None, minr=None, maxr=None, r_scale='linear', nr=nr, nl=nl,
                                  orientation='vertical', subset='short', force_lambda_z=True)

Perform Voronoi binning of orbits in the radius-circularity phase space, based on the best-fitting model so far. The goal is to group the original
``n_orbits`` orbit bundles into fewer ``n_bundle`` Voronoi bundles with each of these Voronoi bundles accounting for
a weight of at least ``vor_weight``.

Note that each original orbit bundle is just one orbit if ``dithering=1``, but an actual bundle if ``dithering > 1``.

The result is a tuple ``(vor_bundle_mapping, phase_space_binning)``:
```
vor_bundle_mapping :    np.array of shape (n_bundle, n_orbits)
                        Mapping between the "original" orbit bundles and the Voronoi
                        orbit bundles: vor_bundle_mapping(i_bundle, i_orbit) is the
                        fraction of i_orbit assigned to i_bundle, multiplied by i_orbit's weight.
phase_space_binning :   dict
                        'in':   np.array of shape (3, nr*nl), the binning input:
                                bin r, bin lambda_z, bin total weight
                        'out':  np.array of shape (3, n_bundle), the Voronoi binning output:
                                weighted Voronoi bin centroid coordinates r_bar, lambda_bar
                                and Voronoi bin total weights
                        'map':  np.array of shape (nr*nl,) the phase space mapping:
                                Voronoi bin numbers for each input bin
```

In [ ]:
coloring = dyn.coloring.Coloring(c)  # the phase space binning resides in the Coloring class (might move to Analysis)
vor_weight = 0.05  # define the desired (minimum) total orbital weight in each Voronoi bin
vor_bundle_mapping, phase_space_binning = coloring.bin_phase_space(model=None,
                                                                   minr='auto',
                                                                   maxr='auto',
                                                                   r_scale='linear',
                                                                   nr=nr,
                                                                   nl=nl,
                                                                   vor_weight=vor_weight,
                                                                   vor_ignore_zeros=False,
                                                                   make_diagnostic_plots=True,
                                                                   extra_diagnostic_output=True,
                                                                   cvt=False,
                                                                   wvt=False)

Let's see how much weight each orbit contributes to each orbit bundle. If ``dithering=1``, then each orbit will contribute its weight to exactly one Voronoi bundle. If ``dithering > 1``, then each orbit actually represents a bundle by itself and can hence contribute to multiple (neighboring) Voronoi orbit bundles.

In [ ]:
# If dithering > 1, orbit bundles may contribute to multiple Voronoi orbit bundles
plt.figure(figsize=(24,4))
plt.gca().set_title('Weight that each orbit bundle contributes to Voronoi orbit bundles')
plt.pcolormesh(np.log10(vor_bundle_mapping), shading='flat', cmap='Greys')
plt.xlabel('Orbit bundle id')
plt.ylabel('Voronoi orbit bundle id')
plt.colorbar(label='log Weight')

Generate maps for each Voronoi orbit bundle. These are mass maps for mass-weighted and flux maps for light-weighted models and plot the mass/light contributed by the individual Voronoi orbital bundles at each aperture.

In [ ]:
a = dyn.analysis.Analysis(c)  # orbit bundle maps are residing in the Analysis class

In [ ]:
bundle_maps = a.get_flux_for_orbit_bundles(pop_set=0, bundle_mapping=vor_bundle_mapping, sb_maps=True)  # might rename the method to get_orbit_bundle_maps()

Orbit bundle maps:
![sb_map](FCC167/output/plots/bundle_sb_maps_pop_pop1.png)

## Observed data

Let's have a look at the observed data...

In [ ]:
stars = c.system.get_unique_triaxial_visible_component()
pops = stars.population_data[0]
#age, dage, met, dmet = np.array([pops.get_data()[i] for i in ('age', 'dage', 'met', 'dmet')])
map_plotter = pops.get_map_plotter()
# map_plotter(age, cmap=cmasher.get_sub_cmap('twilight_shifted',
#                                                   0.05,
#                                                   0.6))
fig = plt.figure(figsize=(20, 10))
fig.subplots_adjust(wspace=0.5)
for i, col in enumerate(('age', 'dage', 'met', 'dmet')):
    ax = plt.subplot(1, 4, i + 1)
    dat = pops.get_data()[col]
    map_plotter(dat,
                vmin=min(dat),
                vmax=max(dat),
                label=col,
                colorbar=True,
                cmap=cmasher.get_sub_cmap('twilight_shifted', 0.05, 0.6))
    ax.set_title(f'{col}')
    ax.set_xlabel('x [arcsec]')
    ax.set_ylabel('y [arcsec]')

# Appendix

## Testing the orbit binning
As dithering=1, each "original" orbit bundle is just one orbit. Hence, 100% of each such orbit bundle will lie in a unique $r, \lambda_z$ input bin and the `vor_bundle_mapping` will map these "original" orbit bundles to the Voronoi orbit bundles.

To test the orbit binning, we will - for each Voronoi orbit bundle - identify the connected "original" orbit bundles and add up their weighted fractions. These should be the same whether computed (a) from the binning input data, (b) from the total weights in the Voronoi bins, and (c) from adding all "original" orbit bundles' contributions in the `vor_bundle_mapping`.

In [ ]:
for vor_bin in range(phase_space_binning['out'].shape[-1]):  # do the following for each Voronoi bin

    input_bins = np.where(phase_space_binning['map']==vor_bin)[0]
    print(f'\ninput bins mapped to {vor_bin=}: {input_bins}')

    # orbit weights in input bins:
    weight_in = [phase_space_binning['in'][2][i] for i in input_bins]  # phase_space_binning['in'].shape=(3, nr*nl), holds r, l, w for each input bin
    weight_in_total = sum(weight_in)
    print(f'{weight_in=}, {weight_in_total=}')

    vorbin_weight = phase_space_binning['out'][2][vor_bin]  # phase_space_binning['out'].shape=(3, n_bundle), holds r, l, w for each Voronoi bin
    print(f'{vorbin_weight     = }')

    vor_bundle_weight = np.sum(vor_bundle_mapping[vor_bin])  # vor_bundle_mapping.shape=(n_bundle, n_orbits), contribution of orbit-weights to Voronoi bins
    print(f'{vor_bundle_weight = }')

    print('Test succeeded? ', np.isclose(weight_in_total, vorbin_weight) and np.isclose(weight_in_total, vor_bundle_weight))

In [ ]:
stars.population_data

In [ ]:
print(flux.shape, np.min(flux), np.max(flux))
minf = min(np.array(list(map(np.log10, flux[grid[s]] / max(flux)))))
maxf = max(np.array(list(map(np.log10, flux[grid[s]] / max(flux)))))
minf, maxf

In [ ]:
c_data = np.array(list(map(np.log10, flux[grid[s]] / max(flux))))
c_data.shape, min(c_data), max(c_data)

In [ ]:
from plotbin import display_pixels
import cmasher as cmr

In [ ]:
a = dyn.analysis.Analysis(c)

In [ ]:
 model_gh_coef = a.get_gh_model_kinematic_maps(v_sigma_option='fit')